<a href="https://colab.research.google.com/github/purpleiron/MySchoolProjects/blob/main/ICT303_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  **ICT303 - Assignment 1**

**Your name: <enter here your full name>**

**Student ID: <enter here your student ID>**

**Email: <enter here your email address>**


## **1. Description**

We would like to develop, using Multilayer Perceptron (MLP), a computer program that takes images of handwritten text, finds the written characters in the image and displays the written characters.

To achieve this, we will proceed in steps:

1. Develop and train an MLP for the recognition of handwritten characters from images. In the first instance, the images are assumed to contain only one handwritten.
2. Train and test the MLP, and evaluate its performance by using loss curves and proper accuracy/performance measures
3. Improve the performance of the MLP by tuning its hyper parameters.
4. Extend the program you developed to localize (detect) and recognize handwritten characters in an image that contains multiple handwritten characters.

For this purpose, we will use the following dataset for training, validation and testing: https://www.kaggle.com/datasets/dhruvildave/english-handwritten-characters-dataset.

You are required to justify every design choice. Justifications should be theoretical and validated with experiments.

It is important that you start as earlier as possible. Coding is usually easy. However, training neural networks and tuning its hyper-parameters takes time.

##**2. Marking Guide**##

- The overal structure of the program - it should follow the structure we used so far in the labs **[30 Marks]**. This includes:
  - A class that defines the network architecture that extends the class `nn.Module`. It should have a constructor method (`__init__()`) and a forward function (`forward()`)
  - The Trainer class
  - A main function

- Training working and running on GPU **[10 marks]**

- Curves for training loss and validation loss plotted and training stopped when the network starts to overfit (i.e., when the validation loss starts to increase). You must use TensorBoard to visualize curves and monitor performance **[10 marks]**

- Testing code properly working. **[10 marks]**

- Hyper parameters finetuned and the best ones selected. **[10 marks]**

- Quality of the dicussions **[20 marks]**: did the student discuss various design choices, including the hyperparamters or any choices they made to improve the performance? Any design choice should be properly justified.

- Extension to the localization of the characters **[10 marks]**

## **3. What to submit**

You need to upload to LMS the notebook as well as a folder that contains the .py files you created. All classes should be implemented in .py files. The notebook will sever as a documentation of your work as well as the codes that demonstrated the training, validation and testing of your MLP models that you created.




#Import necessary dependencies

In [8]:
# Import necessary libraries
import torch
from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import random_split
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import matplotlib.pyplot as plt
import zipfile


import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader

from collections import Counter

#print('CUDA available:', torch.cuda.is_available())
#print('GPU:', torch.cuda.get_device_name(0))

from google.colab import drive
drive.mount('/content/drive')
with zipfile.ZipFile("/content/drive/My Drive/archive.zip", 'r') as zip_ref:
  zip_ref.extractall("/content/dataset_folder")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Data Transformation/preprocessing

I took a look at the images in the dataset and they were black and white. This should allow me to convert the images to greyscale and save on training time. I'm not sure how good it will be if the test images will not be in grayscale, but I can make the test images grayscale as well. If you want to transform the images to a different resolution, just change the parameters in transforms.Resize()

#Load Dataset

#Defining the model



In [9]:
class MLP(nn.Module):
    def __init__(self, input_size=224*224, output_size=62, lr=0.001):
        super(MLP, self).__init__()

        # The output size is 62, which might correspond to the number of classes you have (10 digits + 52 letters).

        self.layers = nn.Sequential(

            nn.Flatten(),

            nn.Linear(input_size, 512),
            # Justification: The first hidden layer has 512 neurons. This number is chosen to provide a
            # good balance between model complexity and computational efficiency.

            nn.ReLU(),
            # Justification: ReLU is used as the activation function because it helps with faster convergence
            # and alleviates the vanishing gradient problem compared to sigmoid or tanh.

            nn.Linear(512, 256),
            # Justification: A second hidden layer with 256 neurons is used to increase the model's ability to
            # capture non-linear relationships in the data.

            nn.ReLU(),
            # Justification: Another ReLU for non-linear activation.

            nn.Linear(256, output_size),
            # Justification: The output layer size corresponds to the number of classes.

            nn.LogSoftmax(dim=1)
            # Justification: LogSoftmax is used in the output layer to obtain log-probabilities which are more
            # numerically stable for the subsequent calculation of the negative log likelihood loss during training.


        )

        self.lr = lr

    def forward(self, x):
        return self.layers(x)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), self.lr)
        # Justification: The Adam optimizer is used as it combines the best properties of the AdaGrad and RMSProp
        # algorithms to provide an optimization algorithm that can handle sparse gradients on noisy problems.

    def loss(self, y_hat, y):
        fn = nn.NLLLoss()
        return fn(y_hat, y)
        # Justification: NLLLoss (Negative Log Likelihood Loss) is used as the loss function for multi-class
        # classification problems when combined with LogSoftmax in the output layer. It is efficient and
        # calculates the loss between the predicted log-probabilities and the ground truth labels.




#Defining the Trainer class

In [10]:
class Trainer:
    def __init__(self, model, optimizer, criterion, train_loader, val_loader=None, num_epochs=25, patience=5, device='cuda'):
        """
        Initialize the Trainer with model, optimizer, criterion, data loaders, and training configurations.
        """
        # Justification for changes:
        # - Added model, optimizer, criterion as parameters for flexibility and explicitness.
        # - Included train_loader and val_loader for separate training and validation data handling.
        # - Added device parameter for flexibility between CPU and GPU training.
        self.model = model.to(device)
        self.optimizer = optimizer
        self.criterion = criterion
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.num_epochs = num_epochs
        self.patience = patience
        self.device = device
        self.best_val_loss = float('inf')
        self.patience_counter = 0
        self.writer = SummaryWriter()  # Default log_dir is fine; no need to customize without specific requirement.

    def train_epoch(self):
        """
        Train the model for one epoch.
        """
        self.model.train()
        running_loss = 0.0
        for inputs, labels in tqdm(self.train_loader, desc="Training"):
            inputs, labels = inputs.to(self.device), labels.to(self.device)
            self.optimizer.zero_grad()
            outputs = self.model(inputs)
            loss = self.criterion(outputs, labels)
            loss.backward()
            self.optimizer.step()
            running_loss += loss.item()
        average_loss = running_loss / len(self.train_loader)
        self.writer.add_scalar('train_loss', average_loss)
        return average_loss

    def validate(self):
        """
        Validate the model on the validation dataset.
        """
        self.model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in tqdm(self.val_loader, desc="Validation"):
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                val_loss += loss.item()
        average_val_loss = val_loss / len(self.val_loader)
        self.writer.add_scalar('val_loss', average_val_loss)
        return average_val_loss

    def fit(self):
        """
        Fit the model to the data.
        """
        for epoch in range(self.num_epochs):
            train_loss = self.train_epoch()
            print(f'Epoch {epoch+1}/{self.num_epochs}, Train Loss: {train_loss:.4f}')

            if self.val_loader:
                val_loss = self.validate()
                print(f'Epoch {epoch+1}/{self.num_epochs}, Validation Loss: {val_loss:.4f}')

                if val_loss < self.best_val_loss:
                    self.best_val_loss = val_loss
                    self.patience_counter = 0
                else:
                    self.patience_counter += 1
                    if self.patience_counter >= self.patience:
                        print('Early stopping triggered')
                        break
        self.writer.close()



#Main function

In [14]:
import torch
from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import os
from collections import Counter

# Your MLP and Trainer classes would be defined here...

def main():
    # Data loading and transformation
    dataset_path = '/content/dataset_folder/archive/Img/'
    transform = transforms.Compose([
        transforms.Grayscale(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])

    # Create the dataset using the ImageFolder wrapper
    dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

    # Map indices to actual class labels
    idx_to_class = {i: cls for cls, i in dataset.class_to_idx.items()}

    # Define the actual classes in order
    actual_classes = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                      'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
                      'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

    # Map the folder indices to actual class labels
    folder_idx_to_label = {idx: label for idx, label in enumerate(actual_classes)}

    # Splitting dataset into training, validation, and test sets
    train_size = int(0.7 * len(dataset))
    val_size = int(0.15 * len(dataset))
    test_size = len(dataset) - train_size - val_size
    train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=64)
    test_loader = DataLoader(test_dataset, batch_size=64)

    # Model initialization
    model = MLP(input_size=224*224, output_size=62, lr=0.001)
    criterion = nn.CrossEntropyLoss()  # No need to apply LogSoftmax in the last layer of your model when using this loss
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


    # Trainer initialization and training
    trainer = Trainer(
        model=model,
        optimizer=optimizer,
        criterion=criterion,
        train_loader=train_loader,
        val_loader=val_loader,
        num_epochs=40,
        patience=5,
        device='cuda'
    )
    trainer.fit()

    # Test the model
    model.eval()  # Set the model to evaluation mode
    test_predictions, test_labels = [], []

    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc="Testing"):
            inputs = inputs.to('cuda')
            labels = labels.to('cuda')
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            test_predictions.extend(predicted.view(-1).tolist())
            test_labels.extend(labels.view(-1).tolist())

    # Calculate the accuracy
    accuracy = accuracy_score(test_labels, test_predictions)
    print(f"Test Accuracy: {accuracy:.4f}")

    # Optionally, print out a few predictions
    for i in range(10):
        print(f"True label: {folder_idx_to_label[test_labels[i]]}, Predicted label: {folder_idx_to_label[test_predictions[i]]}")


#Running

In [15]:
if __name__ == "__main__":
    main()

Training: 100%|██████████| 38/38 [00:38<00:00,  1.01s/it]


Epoch 1/40, Train Loss: 4.1411


Validation: 100%|██████████| 8/8 [00:08<00:00,  1.02s/it]


Epoch 1/40, Validation Loss: 4.1455


Training: 100%|██████████| 38/38 [00:36<00:00,  1.04it/s]


...

Epoch 40/40, Train Loss: 2.5472


Validation: 100%|██████████| 8/8 [00:08<00:00,  1.02s/it]


Epoch 40/40, Validation Loss: 2.9211


Testing: 100%|██████████| 8/8 [00:07<00:00,  1.10it/s]

Test Accuracy: 0.2812
True label: R, Predicted label: G
True label: R, Predicted label: G
True label: z, Predicted label: L
True label: 5, Predicted label: G
True label: W, Predicted label: m
True label: q, Predicted label: e
True label: L, Predicted label: i
True label: 5, Predicted label: S
True label: Z, Predicted label: Q
True label: Q, Predicted label: W


In [ ]:

'''
import torch
from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import os
from collections import Counter

# Your MLP and Trainer classes would be defined here...

def main():
    # Data loading and transformation
    dataset_path = '/content/dataset_folder/archive/Img/'
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])

    # Create the dataset using the ImageFolder wrapper
    dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

    # Map indices to actual class labels
    idx_to_class = {i: cls for cls, i in dataset.class_to_idx.items()}

    # Define the actual classes in order
    actual_classes = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                      'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
                      'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

    # Map the folder indices to actual class labels
    folder_idx_to_label = {idx: label for idx, label in enumerate(actual_classes)}

    # Splitting dataset into training, validation, and test sets
    train_size = int(0.7 * len(dataset))
    val_size = int(0.15 * len(dataset))
    test_size = len(dataset) - train_size - val_size
    train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=64)
    test_loader = DataLoader(test_dataset, batch_size=64)

    # Model initialization
    model = MLP(input_size=224*224*3, output_size=62, lr=0.001)
    criterion = nn.NLLLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Trainer initialization and training
    trainer = Trainer(
        model=model,
        optimizer=optimizer,
        criterion=criterion,
        train_loader=train_loader,
        val_loader=val_loader,
        num_epochs=25,
        patience=5,
        device='cuda'
    )
    trainer.fit()

    # Test the model
    model.eval()  # Set the model to evaluation mode
    test_predictions, test_labels = [], []

    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc="Testing"):
            inputs = inputs.to('cuda')
            labels = labels.to('cuda')
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            test_predictions.extend(predicted.view(-1).tolist())
            test_labels.extend(labels.view(-1).tolist())

    # Calculate the accuracy
    accuracy = accuracy_score(test_labels, test_predictions)
    print(f"Test Accuracy: {accuracy:.4f}")

    # Optionally, print out a few predictions
    for i in range(10):
        print(f"True label: {folder_idx_to_label[test_labels[i]]}, Predicted label: {folder_idx_to_label[test_predictions[i]]}")
'''

#First test
This is using 224x224x3. It seems pretty bad, I'm gonna try to preprocess the photos to be grayscale so that there would be 3 times lesser inputs first


```
Training: 100%|██████████| 38/38 [00:43<00:00,  1.13s/it]
Epoch 1/25, Train Loss: 22.8603
Validation: 100%|██████████| 8/8 [00:09<00:00,  1.16s/it]
Epoch 1/25, Validation Loss: 4.6418
Training: 100%|██████████| 38/38 [00:43<00:00,  1.14s/it]
Epoch 2/25, Train Loss: 4.2615
Validation: 100%|██████████| 8/8 [00:09<00:00,  1.16s/it]
Epoch 2/25, Validation Loss: 4.1615
Training: 100%|██████████| 38/38 [00:43<00:00,  1.13s/it]
Epoch 3/25, Train Loss: 4.1377
Validation: 100%|██████████| 8/8 [00:09<00:00,  1.18s/it]
Epoch 3/25, Validation Loss: 4.1130
Training: 100%|██████████| 38/38 [00:44<00:00,  1.17s/it]
Epoch 4/25, Train Loss: 4.1241
Validation: 100%|██████████| 8/8 [00:09<00:00,  1.18s/it]
Epoch 4/25, Validation Loss: 4.1286
Training: 100%|██████████| 38/38 [00:43<00:00,  1.14s/it]
Epoch 5/25, Train Loss: 4.1279
Validation: 100%|██████████| 8/8 [00:09<00:00,  1.18s/it]
Epoch 5/25, Validation Loss: 4.1297
Training: 100%|██████████| 38/38 [00:43<00:00,  1.15s/it]
Epoch 6/25, Train Loss: 4.1275
Validation: 100%|██████████| 8/8 [00:09<00:00,  1.13s/it]
Epoch 6/25, Validation Loss: 4.1305
Training: 100%|██████████| 38/38 [00:43<00:00,  1.16s/it]
Epoch 7/25, Train Loss: 4.1270
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.10s/it]
Epoch 7/25, Validation Loss: 4.1313
Training: 100%|██████████| 38/38 [00:44<00:00,  1.17s/it]
Epoch 8/25, Train Loss: 4.1268
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.05s/it]
Epoch 8/25, Validation Loss: 4.1323
Early stopping triggered
Testing: 100%|██████████| 8/8 [00:09<00:00,  1.18s/it]Test Accuracy: 0.0117
True label: 2, Predicted label: c
True label: P, Predicted label: c
True label: s, Predicted label: c
True label: E, Predicted label: c
True label: U, Predicted label: c
True label: o, Predicted label: c
True label: 6, Predicted label: c
True label: t, Predicted label: c
True label: Q, Predicted label: c
True label: 8, Predicted label: c
```



#Second try
I changed it to grayscale to see if it would be better or be faster but I don't think it did much. For some reason, both time the model predicted the same thing the whole time



```
Training: 100%|██████████| 38/38 [00:43<00:00,  1.15s/it]
Epoch 1/25, Train Loss: 35.9448
Validation: 100%|██████████| 8/8 [00:09<00:00,  1.24s/it]
Epoch 1/25, Validation Loss: 5.5344
Training: 100%|██████████| 38/38 [00:43<00:00,  1.14s/it]
Epoch 2/25, Train Loss: 4.2988
Validation: 100%|██████████| 8/8 [00:09<00:00,  1.18s/it]
Epoch 2/25, Validation Loss: 4.1325
Training: 100%|██████████| 38/38 [00:43<00:00,  1.14s/it]
Epoch 3/25, Train Loss: 4.1148
Validation: 100%|██████████| 8/8 [00:09<00:00,  1.22s/it]
Epoch 3/25, Validation Loss: 4.1123
Training: 100%|██████████| 38/38 [00:43<00:00,  1.15s/it]
Epoch 4/25, Train Loss: 4.1165
Validation: 100%|██████████| 8/8 [00:09<00:00,  1.19s/it]
Epoch 4/25, Validation Loss: 4.1284
Training: 100%|██████████| 38/38 [00:43<00:00,  1.16s/it]
Epoch 5/25, Train Loss: 4.1276
Validation: 100%|██████████| 8/8 [00:09<00:00,  1.20s/it]
Epoch 5/25, Validation Loss: 4.1291
Training: 100%|██████████| 38/38 [00:44<00:00,  1.17s/it]
Epoch 6/25, Train Loss: 4.1272
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.07s/it]
Epoch 6/25, Validation Loss: 4.1302
Training: 100%|██████████| 38/38 [00:48<00:00,  1.27s/it]
Epoch 7/25, Train Loss: 4.1266
Validation: 100%|██████████| 8/8 [00:10<00:00,  1.35s/it]
Epoch 7/25, Validation Loss: 4.1312
Training: 100%|██████████| 38/38 [00:56<00:00,  1.48s/it]
Epoch 8/25, Train Loss: 4.1265
Validation: 100%|██████████| 8/8 [00:11<00:00,  1.47s/it]
Epoch 8/25, Validation Loss: 4.1321
Early stopping triggered
Testing: 100%|██████████| 8/8 [00:09<00:00,  1.23s/it]Test Accuracy: 0.0059
True label: V, Predicted label: 6
True label: 8, Predicted label: 6
True label: K, Predicted label: 6
True label: u, Predicted label: 6
True label: o, Predicted label: 6
True label: X, Predicted label: 6
True label: 1, Predicted label: 6
True label: D, Predicted label: 6
True label: 9, Predicted label: 6
True label: 8, Predicted label: 6
```



# Third try
Seems like I didn't do the grayscale properly, so now I added

```
transforms.Grayscale(),
```
to the transform part of the main function. Then I tried again:



```
Training: 100%|██████████| 38/38 [00:36<00:00,  1.03it/s]
Epoch 1/25, Train Loss: 8.7023
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.11it/s]
Epoch 1/25, Validation Loss: 4.2525
Training: 100%|██████████| 38/38 [00:36<00:00,  1.04it/s]
Epoch 2/25, Train Loss: 4.1504
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.08s/it]
Epoch 2/25, Validation Loss: 4.1328
Training: 100%|██████████| 38/38 [00:46<00:00,  1.21s/it]
Epoch 3/25, Train Loss: 4.0829
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.12s/it]
Epoch 3/25, Validation Loss: 4.1511
Training: 100%|██████████| 38/38 [00:43<00:00,  1.14s/it]
Epoch 4/25, Train Loss: 3.9323
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.05it/s]
Epoch 4/25, Validation Loss: 3.9317
Training: 100%|██████████| 38/38 [00:36<00:00,  1.04it/s]
Epoch 5/25, Train Loss: 3.7808
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.02s/it]
Epoch 5/25, Validation Loss: 3.8885
Training: 100%|██████████| 38/38 [00:36<00:00,  1.04it/s]
Epoch 6/25, Train Loss: 3.6409
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.11it/s]
Epoch 6/25, Validation Loss: 3.9240
Training: 100%|██████████| 38/38 [00:36<00:00,  1.03it/s]
Epoch 7/25, Train Loss: 3.4636
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.02s/it]
Epoch 7/25, Validation Loss: 3.5234
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 8/25, Train Loss: 3.3190
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.02it/s]
Epoch 8/25, Validation Loss: 3.5741
Training: 100%|██████████| 38/38 [00:36<00:00,  1.04it/s]
Epoch 9/25, Train Loss: 3.1756
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.08it/s]
Epoch 9/25, Validation Loss: 3.5298
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 10/25, Train Loss: 3.0995
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.02s/it]
Epoch 10/25, Validation Loss: 3.2305
Training: 100%|██████████| 38/38 [00:36<00:00,  1.03it/s]
Epoch 11/25, Train Loss: 2.9847
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.12it/s]
Epoch 11/25, Validation Loss: 3.4461
Training: 100%|██████████| 38/38 [00:36<00:00,  1.04it/s]
Epoch 12/25, Train Loss: 2.8964
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.01s/it]
Epoch 12/25, Validation Loss: 3.7214
Training: 100%|██████████| 38/38 [00:35<00:00,  1.06it/s]
Epoch 13/25, Train Loss: 2.8732
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.01s/it]
Epoch 13/25, Validation Loss: 3.0940
Training: 100%|██████████| 38/38 [00:36<00:00,  1.04it/s]
Epoch 14/25, Train Loss: 2.6759
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.11it/s]
Epoch 14/25, Validation Loss: 3.2202
Training: 100%|██████████| 38/38 [00:36<00:00,  1.03it/s]
Epoch 15/25, Train Loss: 2.6783
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.02s/it]
Epoch 15/25, Validation Loss: 3.0593
Training: 100%|██████████| 38/38 [00:35<00:00,  1.06it/s]
Epoch 16/25, Train Loss: 2.5362
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.04it/s]
Epoch 16/25, Validation Loss: 2.9889
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 17/25, Train Loss: 2.4245
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.12it/s]
Epoch 17/25, Validation Loss: 2.8331
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 18/25, Train Loss: 2.4480
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.00it/s]
Epoch 18/25, Validation Loss: 2.8302
Training: 100%|██████████| 38/38 [00:35<00:00,  1.07it/s]
Epoch 19/25, Train Loss: 2.2620
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.05it/s]
Epoch 19/25, Validation Loss: 2.8571
Training: 100%|██████████| 38/38 [00:35<00:00,  1.06it/s]
Epoch 20/25, Train Loss: 2.2319
Validation: 100%|██████████| 8/8 [00:06<00:00,  1.15it/s]
Epoch 20/25, Validation Loss: 2.9226
Training: 100%|██████████| 38/38 [00:35<00:00,  1.06it/s]
Epoch 21/25, Train Loss: 2.0822
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.01it/s]
Epoch 21/25, Validation Loss: 2.8602
Training: 100%|██████████| 38/38 [00:35<00:00,  1.08it/s]
Epoch 22/25, Train Loss: 2.0601
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.03it/s]
Epoch 22/25, Validation Loss: 2.7978
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 23/25, Train Loss: 2.0359
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.14it/s]
Epoch 23/25, Validation Loss: 2.8043
Training: 100%|██████████| 38/38 [00:35<00:00,  1.06it/s]
Epoch 24/25, Train Loss: 1.9455
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.00it/s]
Epoch 24/25, Validation Loss: 2.6880
Training: 100%|██████████| 38/38 [00:35<00:00,  1.08it/s]
Epoch 25/25, Train Loss: 2.0011
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.01it/s]
Epoch 25/25, Validation Loss: 2.7536
Testing: 100%|██████████| 8/8 [00:07<00:00,  1.03it/s]Test Accuracy: 0.3613
True label: v, Predicted label: v
True label: r, Predicted label: r
True label: D, Predicted label: D
True label: A, Predicted label: a
True label: G, Predicted label: G
True label: j, Predicted label: j
True label: z, Predicted label: y
True label: l, Predicted label: l
True label: e, Predicted label: y
True label: 3, Predicted label: X

```

This time it was much better, it was able to finish training without stopping early and the accuracy was much better than the previous times. I think I might change the learning rate next.


#Fourth try

I changed the learning rate to 0.03. It was still able to finish without early stopping, but did not perform better than the one with 0.001 learning rate



```
Training: 100%|██████████| 38/38 [00:35<00:00,  1.07it/s]
Epoch 1/25, Train Loss: 7.3398
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.02it/s]
Epoch 1/25, Validation Loss: 4.2296
Training: 100%|██████████| 38/38 [00:35<00:00,  1.06it/s]
Epoch 2/25, Train Loss: 4.1507
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.14it/s]
Epoch 2/25, Validation Loss: 4.1157
Training: 100%|██████████| 38/38 [00:36<00:00,  1.04it/s]
Epoch 3/25, Train Loss: 4.1202
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.05it/s]
Epoch 3/25, Validation Loss: 4.1253
Training: 100%|██████████| 38/38 [00:35<00:00,  1.08it/s]
Epoch 4/25, Train Loss: 4.1133
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.00s/it]
Epoch 4/25, Validation Loss: 4.1359
Training: 100%|██████████| 38/38 [00:35<00:00,  1.06it/s]
Epoch 5/25, Train Loss: 4.1178
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.09it/s]
Epoch 5/25, Validation Loss: 4.1221
Training: 100%|██████████| 38/38 [00:35<00:00,  1.06it/s]
Epoch 6/25, Train Loss: 4.1093
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.09it/s]
Epoch 6/25, Validation Loss: 4.1197
Training: 100%|██████████| 38/38 [00:35<00:00,  1.06it/s]
Epoch 7/25, Train Loss: 4.1082
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.00it/s]
Epoch 7/25, Validation Loss: 4.1089
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 8/25, Train Loss: 4.1089
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.14it/s]
Epoch 8/25, Validation Loss: 4.1131
Training: 100%|██████████| 38/38 [00:36<00:00,  1.03it/s]
Epoch 9/25, Train Loss: 4.0958
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.04it/s]
Epoch 9/25, Validation Loss: 4.1130
Training: 100%|██████████| 38/38 [00:35<00:00,  1.06it/s]
Epoch 10/25, Train Loss: 4.0435
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.00s/it]
Epoch 10/25, Validation Loss: 4.0844
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 11/25, Train Loss: 3.9772
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.13it/s]
Epoch 11/25, Validation Loss: 4.0009
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 12/25, Train Loss: 3.9166
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.05it/s]
Epoch 12/25, Validation Loss: 4.0524
Training: 100%|██████████| 38/38 [00:35<00:00,  1.07it/s]
Epoch 13/25, Train Loss: 3.8353
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.01it/s]
Epoch 13/25, Validation Loss: 3.7611
Training: 100%|██████████| 38/38 [00:35<00:00,  1.06it/s]
Epoch 14/25, Train Loss: 3.7217
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.12it/s]
Epoch 14/25, Validation Loss: 3.6748
Training: 100%|██████████| 38/38 [00:35<00:00,  1.06it/s]
Epoch 15/25, Train Loss: 3.6518
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.10it/s]
Epoch 15/25, Validation Loss: 3.4618
Training: 100%|██████████| 38/38 [00:35<00:00,  1.07it/s]
Epoch 16/25, Train Loss: 3.4461
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.01it/s]
Epoch 16/25, Validation Loss: 3.3203
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 17/25, Train Loss: 3.3810
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.11it/s]
Epoch 17/25, Validation Loss: 3.3261
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 18/25, Train Loss: 3.3177
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.09it/s]
Epoch 18/25, Validation Loss: 3.1893
Training: 100%|██████████| 38/38 [00:35<00:00,  1.06it/s]
Epoch 19/25, Train Loss: 3.2198
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.01it/s]
Epoch 19/25, Validation Loss: 3.2495
Training: 100%|██████████| 38/38 [00:35<00:00,  1.07it/s]
Epoch 20/25, Train Loss: 3.1197
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.09it/s]
Epoch 20/25, Validation Loss: 3.1028
Training: 100%|██████████| 38/38 [00:35<00:00,  1.06it/s]
Epoch 21/25, Train Loss: 3.0152
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.13it/s]
Epoch 21/25, Validation Loss: 3.0579
Training: 100%|██████████| 38/38 [00:35<00:00,  1.06it/s]
Epoch 22/25, Train Loss: 2.9061
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.02it/s]
Epoch 22/25, Validation Loss: 2.9536
Training: 100%|██████████| 38/38 [00:35<00:00,  1.07it/s]
Epoch 23/25, Train Loss: 2.8616
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.06it/s]
Epoch 23/25, Validation Loss: 3.0566
Training: 100%|██████████| 38/38 [00:36<00:00,  1.04it/s]
Epoch 24/25, Train Loss: 2.7837
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.09it/s]
Epoch 24/25, Validation Loss: 3.0775
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 25/25, Train Loss: 2.7174
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.01it/s]
Epoch 25/25, Validation Loss: 2.8468
Testing: 100%|██████████| 8/8 [00:06<00:00,  1.15it/s]Test Accuracy: 0.2812
True label: O, Predicted label: O
True label: W, Predicted label: W
True label: y, Predicted label: c
True label: W, Predicted label: U
True label: 3, Predicted label: J
True label: t, Predicted label: b
True label: b, Predicted label: U
True label: r, Predicted label: N
True label: o, Predicted label: Y
True label: D, Predicted label: V
```



#Fifth try

I think I will reduce learning rate to 0.02 and then see how it fares.



```
Training: 100%|██████████| 38/38 [00:36<00:00,  1.03it/s]
Epoch 1/25, Train Loss: 8.8465
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.01s/it]
Epoch 1/25, Validation Loss: 4.2102
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 2/25, Train Loss: 4.1448
Validation: 100%|██████████| 8/8 [00:06<00:00,  1.14it/s]
Epoch 2/25, Validation Loss: 4.1407
Training: 100%|██████████| 38/38 [00:38<00:00,  1.02s/it]
Epoch 3/25, Train Loss: 4.1249
Validation: 100%|██████████| 8/8 [00:10<00:00,  1.37s/it]
Epoch 3/25, Validation Loss: 4.1327
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 4/25, Train Loss: 4.1280
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.13it/s]
Epoch 4/25, Validation Loss: 4.1348
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 5/25, Train Loss: 4.1323
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.02s/it]
Epoch 5/25, Validation Loss: 4.1310
Training: 100%|██████████| 38/38 [00:36<00:00,  1.06it/s]
Epoch 6/25, Train Loss: 4.1267
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.00it/s]
Epoch 6/25, Validation Loss: 4.1314
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 7/25, Train Loss: 4.1265
Validation: 100%|██████████| 8/8 [00:06<00:00,  1.15it/s]
Epoch 7/25, Validation Loss: 4.1320
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 8/25, Train Loss: 4.1263
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.00s/it]
Epoch 8/25, Validation Loss: 4.1325
Training: 100%|██████████| 38/38 [00:35<00:00,  1.08it/s]
Epoch 9/25, Train Loss: 4.1264
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.00it/s]
Epoch 9/25, Validation Loss: 4.1331
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 10/25, Train Loss: 4.1259
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.13it/s]
Epoch 10/25, Validation Loss: 4.1338
Early stopping triggered
Testing: 100%|██████████| 8/8 [00:08<00:00,  1.01s/it]Test Accuracy: 0.0156
True label: I, Predicted label: N
True label: F, Predicted label: N
True label: J, Predicted label: N
True label: Q, Predicted label: N
True label: y, Predicted label: N
True label: j, Predicted label: N
True label: 1, Predicted label: N
True label: v, Predicted label: N
True label: P, Predicted label: N
True label: 3, Predicted label: N

```



#6th try

I'll bring the lr back to 0.001 and see if it does well again



```
Training: 100%|██████████| 38/38 [00:35<00:00,  1.08it/s]
Epoch 1/25, Train Loss: 9.6670
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.01it/s]
Epoch 1/25, Validation Loss: 4.2237
Training: 100%|██████████| 38/38 [00:36<00:00,  1.03it/s]
Epoch 2/25, Train Loss: 4.1985
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.05it/s]
Epoch 2/25, Validation Loss: 4.1960
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 3/25, Train Loss: 4.1402
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.01it/s]
Epoch 3/25, Validation Loss: 4.1243
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 4/25, Train Loss: 4.1204
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.13it/s]
Epoch 4/25, Validation Loss: 4.1096
Training: 100%|██████████| 38/38 [00:36<00:00,  1.03it/s]
Epoch 5/25, Train Loss: 4.0962
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.04it/s]
Epoch 5/25, Validation Loss: 4.0977
Training: 100%|██████████| 38/38 [00:35<00:00,  1.06it/s]
Epoch 6/25, Train Loss: 4.0595
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.01it/s]
Epoch 6/25, Validation Loss: 4.0675
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 7/25, Train Loss: 3.9946
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.14it/s]
Epoch 7/25, Validation Loss: 4.0190
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 8/25, Train Loss: 3.8856
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.01it/s]
Epoch 8/25, Validation Loss: 3.9141
Training: 100%|██████████| 38/38 [00:35<00:00,  1.08it/s]
Epoch 9/25, Train Loss: 3.7260
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.00s/it]
Epoch 9/25, Validation Loss: 3.7848
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 10/25, Train Loss: 3.5669
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.13it/s]
Epoch 10/25, Validation Loss: 3.6337
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 11/25, Train Loss: 3.5191
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.01it/s]
Epoch 11/25, Validation Loss: 3.5165
Training: 100%|██████████| 38/38 [00:35<00:00,  1.08it/s]
Epoch 12/25, Train Loss: 3.3459
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.00it/s]
Epoch 12/25, Validation Loss: 3.6097
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 13/25, Train Loss: 3.2066
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.14it/s]
Epoch 13/25, Validation Loss: 3.4540
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 14/25, Train Loss: 3.1306
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.00it/s]
Epoch 14/25, Validation Loss: 3.2902
Training: 100%|██████████| 38/38 [00:35<00:00,  1.08it/s]
Epoch 15/25, Train Loss: 3.0170
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.01s/it]
Epoch 15/25, Validation Loss: 3.1501
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 16/25, Train Loss: 2.9013
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.14it/s]
Epoch 16/25, Validation Loss: 3.1092
Training: 100%|██████████| 38/38 [00:35<00:00,  1.06it/s]
Epoch 17/25, Train Loss: 2.8617
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.00s/it]
Epoch 17/25, Validation Loss: 3.2535
Training: 100%|██████████| 38/38 [00:35<00:00,  1.08it/s]
Epoch 18/25, Train Loss: 2.8326
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.01it/s]
Epoch 18/25, Validation Loss: 2.9689
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 19/25, Train Loss: 2.7579
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.14it/s]
Epoch 19/25, Validation Loss: 2.9509
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 20/25, Train Loss: 2.6418
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.00it/s]
Epoch 20/25, Validation Loss: 3.0926
Training: 100%|██████████| 38/38 [00:35<00:00,  1.08it/s]
Epoch 21/25, Train Loss: 2.5455
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.00it/s]
Epoch 21/25, Validation Loss: 2.8725
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 22/25, Train Loss: 2.4774
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.12it/s]
Epoch 22/25, Validation Loss: 2.9118
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 23/25, Train Loss: 2.4733
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.01it/s]
Epoch 23/25, Validation Loss: 2.9952
Training: 100%|██████████| 38/38 [00:35<00:00,  1.07it/s]
Epoch 24/25, Train Loss: 2.3494
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.00it/s]
Epoch 24/25, Validation Loss: 2.8508
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 25/25, Train Loss: 2.3004
Validation: 100%|██████████| 8/8 [00:06<00:00,  1.15it/s]
Epoch 25/25, Validation Loss: 2.7803
Testing: 100%|██████████| 8/8 [00:08<00:00,  1.00s/it]Test Accuracy: 0.2812
True label: T, Predicted label: 7
True label: f, Predicted label: f
True label: z, Predicted label: j
True label: q, Predicted label: 1
True label: 2, Predicted label: l
True label: v, Predicted label: y
True label: W, Predicted label: W
True label: H, Predicted label: H
True label: T, Predicted label: 7
True label: q, Predicted label: g
```



#7th try

For the 7th try, I tried changing the optimizer and the loss function:



```
criterion = nn.CrossEntropyLoss()  # No need to apply LogSoftmax in the last layer of your model when using this loss
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
```

Seems like it worked pretty well, it completed the 25 epochs and got a

```
Training: 100%|██████████| 38/38 [00:42<00:00,  1.12s/it]
Epoch 1/25, Train Loss: 4.1381
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.00s/it]
Epoch 1/25, Validation Loss: 4.1410
Training: 100%|██████████| 38/38 [00:40<00:00,  1.06s/it]
Epoch 2/25, Train Loss: 4.1314
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.01s/it]
Epoch 2/25, Validation Loss: 4.1417
Training: 100%|██████████| 38/38 [00:35<00:00,  1.08it/s]
Epoch 3/25, Train Loss: 4.1244
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.01it/s]
Epoch 3/25, Validation Loss: 4.1456
Training: 100%|██████████| 38/38 [00:36<00:00,  1.04it/s]
Epoch 4/25, Train Loss: 4.1201
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.12it/s]
Epoch 4/25, Validation Loss: 4.1376
Training: 100%|██████████| 38/38 [00:35<00:00,  1.06it/s]
Epoch 5/25, Train Loss: 4.1141
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.02s/it]
Epoch 5/25, Validation Loss: 4.1362
Training: 100%|██████████| 38/38 [00:35<00:00,  1.07it/s]
Epoch 6/25, Train Loss: 4.1088
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.00it/s]
Epoch 6/25, Validation Loss: 4.1209
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 7/25, Train Loss: 4.0994
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.13it/s]
Epoch 7/25, Validation Loss: 4.1268
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 8/25, Train Loss: 4.0902
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.04s/it]
Epoch 8/25, Validation Loss: 4.1169
Training: 100%|██████████| 38/38 [00:35<00:00,  1.08it/s]
Epoch 9/25, Train Loss: 4.0784
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.03it/s]
Epoch 9/25, Validation Loss: 4.1029
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 10/25, Train Loss: 4.0598
Validation: 100%|██████████| 8/8 [00:06<00:00,  1.15it/s]
Epoch 10/25, Validation Loss: 4.0824
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 11/25, Train Loss: 4.0396
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.00s/it]
Epoch 11/25, Validation Loss: 4.0698
Training: 100%|██████████| 38/38 [00:35<00:00,  1.08it/s]
Epoch 12/25, Train Loss: 4.0062
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.00it/s]
Epoch 12/25, Validation Loss: 4.0361
Training: 100%|██████████| 38/38 [00:36<00:00,  1.06it/s]
Epoch 13/25, Train Loss: 3.9735
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.12it/s]
Epoch 13/25, Validation Loss: 4.0084
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 14/25, Train Loss: 3.9209
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.00s/it]
Epoch 14/25, Validation Loss: 3.9489
Training: 100%|██████████| 38/38 [00:35<00:00,  1.06it/s]
Epoch 15/25, Train Loss: 3.8522
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.02s/it]
Epoch 15/25, Validation Loss: 3.8863
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 16/25, Train Loss: 3.7786
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.14it/s]
Epoch 16/25, Validation Loss: 3.8068
Training: 100%|██████████| 38/38 [00:35<00:00,  1.06it/s]
Epoch 17/25, Train Loss: 3.7100
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.00it/s]
Epoch 17/25, Validation Loss: 3.7687
Training: 100%|██████████| 38/38 [00:35<00:00,  1.08it/s]
Epoch 18/25, Train Loss: 3.6240
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.01it/s]
Epoch 18/25, Validation Loss: 3.6588
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 19/25, Train Loss: 3.5370
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.12it/s]
Epoch 19/25, Validation Loss: 3.6351
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 20/25, Train Loss: 3.4779
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.01s/it]
Epoch 20/25, Validation Loss: 3.5935
Training: 100%|██████████| 38/38 [00:35<00:00,  1.07it/s]
Epoch 21/25, Train Loss: 3.3903
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.02it/s]
Epoch 21/25, Validation Loss: 3.5146
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 22/25, Train Loss: 3.3309
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.12it/s]
Epoch 22/25, Validation Loss: 3.4142
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 23/25, Train Loss: 3.2475
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.01s/it]
Epoch 23/25, Validation Loss: 3.4022
Training: 100%|██████████| 38/38 [00:35<00:00,  1.07it/s]
Epoch 24/25, Train Loss: 3.2034
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.04it/s]
Epoch 24/25, Validation Loss: 3.3577
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 25/25, Train Loss: 3.1903
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.14it/s]
Epoch 25/25, Validation Loss: 3.3033
Testing: 100%|██████████| 8/8 [00:08<00:00,  1.01s/it]Test Accuracy: 0.1836
True label: c, Predicted label: c
True label: F, Predicted label: F
True label: C, Predicted label: C
True label: P, Predicted label: R
True label: K, Predicted label: X
True label: b, Predicted label: h
True label: m, Predicted label: m
True label: D, Predicted label: 6
True label: 1, Predicted label: C
True label: m, Predicted label: m
```

And got an accuracy of 18%. I think I will continue using this and change the number of epochs next.



Training: 100%|██████████| 38/38 [00:38<00:00,  1.01s/it]
Epoch 1/40, Train Loss: 4.1411
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.02s/it]
Epoch 1/40, Validation Loss: 4.1455
Training: 100%|██████████| 38/38 [00:36<00:00,  1.04it/s]
Epoch 2/40, Train Loss: 4.1308
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.13it/s]
Epoch 2/40, Validation Loss: 4.1425
Training: 100%|██████████| 38/38 [00:36<00:00,  1.04it/s]
Epoch 3/40, Train Loss: 4.1259
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.01s/it]
Epoch 3/40, Validation Loss: 4.1364
Training: 100%|██████████| 38/38 [00:35<00:00,  1.08it/s]
Epoch 4/40, Train Loss: 4.1210
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.00s/it]
Epoch 4/40, Validation Loss: 4.1381
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 5/40, Train Loss: 4.1139
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.12it/s]
Epoch 5/40, Validation Loss: 4.1286
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 6/40, Train Loss: 4.1049
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.01s/it]
Epoch 6/40, Validation Loss: 4.1222
Training: 100%|██████████| 38/38 [00:35<00:00,  1.06it/s]
Epoch 7/40, Train Loss: 4.0977
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.04it/s]
Epoch 7/40, Validation Loss: 4.1108
Training: 100%|██████████| 38/38 [00:36<00:00,  1.03it/s]
Epoch 8/40, Train Loss: 4.0820
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.07it/s]
Epoch 8/40, Validation Loss: 4.1079
Training: 100%|██████████| 38/38 [00:35<00:00,  1.06it/s]
Epoch 9/40, Train Loss: 4.0645
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.01s/it]
Epoch 9/40, Validation Loss: 4.0861
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 10/40, Train Loss: 4.0428
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.13it/s]
Epoch 10/40, Validation Loss: 4.0572
Training: 100%|██████████| 38/38 [00:36<00:00,  1.03it/s]
Epoch 11/40, Train Loss: 4.0133
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.00s/it]
Epoch 11/40, Validation Loss: 4.0365
Training: 100%|██████████| 38/38 [00:35<00:00,  1.07it/s]
Epoch 12/40, Train Loss: 3.9824
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.02s/it]
Epoch 12/40, Validation Loss: 3.9865
Training: 100%|██████████| 38/38 [00:36<00:00,  1.03it/s]
Epoch 13/40, Train Loss: 3.9277
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.11it/s]
Epoch 13/40, Validation Loss: 3.9485
Training: 100%|██████████| 38/38 [00:36<00:00,  1.04it/s]
Epoch 14/40, Train Loss: 3.8664
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.02s/it]
Epoch 14/40, Validation Loss: 3.8826
Training: 100%|██████████| 38/38 [00:36<00:00,  1.03it/s]
Epoch 15/40, Train Loss: 3.8090
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.09it/s]
Epoch 15/40, Validation Loss: 3.8283
Training: 100%|██████████| 38/38 [00:36<00:00,  1.03it/s]
Epoch 16/40, Train Loss: 3.7227
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.03s/it]
Epoch 16/40, Validation Loss: 3.7315
Training: 100%|██████████| 38/38 [00:35<00:00,  1.06it/s]
Epoch 17/40, Train Loss: 3.6306
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.01s/it]
Epoch 17/40, Validation Loss: 3.6606
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 18/40, Train Loss: 3.5415
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.11it/s]
Epoch 18/40, Validation Loss: 3.5929
Training: 100%|██████████| 38/38 [00:36<00:00,  1.04it/s]
Epoch 19/40, Train Loss: 3.4691
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.02s/it]
Epoch 19/40, Validation Loss: 3.5384
Training: 100%|██████████| 38/38 [00:35<00:00,  1.06it/s]
Epoch 20/40, Train Loss: 3.3881
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.04it/s]
Epoch 20/40, Validation Loss: 3.5274
Training: 100%|██████████| 38/38 [00:36<00:00,  1.03it/s]
Epoch 21/40, Train Loss: 3.3334
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.06it/s]
Epoch 21/40, Validation Loss: 3.4808
Training: 100%|██████████| 38/38 [00:36<00:00,  1.04it/s]
Epoch 22/40, Train Loss: 3.2309
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.02s/it]
Epoch 22/40, Validation Loss: 3.3559
Training: 100%|██████████| 38/38 [00:36<00:00,  1.03it/s]
Epoch 23/40, Train Loss: 3.1966
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.12it/s]
Epoch 23/40, Validation Loss: 3.2991
Training: 100%|██████████| 38/38 [00:36<00:00,  1.03it/s]
Epoch 24/40, Train Loss: 3.1300
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.02s/it]
Epoch 24/40, Validation Loss: 3.2419
Training: 100%|██████████| 38/38 [00:35<00:00,  1.06it/s]
Epoch 25/40, Train Loss: 3.0773
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.02it/s]
Epoch 25/40, Validation Loss: 3.3151
Training: 100%|██████████| 38/38 [00:36<00:00,  1.04it/s]
Epoch 26/40, Train Loss: 3.0425
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.09it/s]
Epoch 26/40, Validation Loss: 3.1774
Training: 100%|██████████| 38/38 [00:36<00:00,  1.04it/s]
Epoch 27/40, Train Loss: 2.9675
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.03s/it]
Epoch 27/40, Validation Loss: 3.1693
Training: 100%|██████████| 38/38 [00:36<00:00,  1.03it/s]
Epoch 28/40, Train Loss: 2.9717
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.11it/s]
Epoch 28/40, Validation Loss: 3.1792
Training: 100%|██████████| 38/38 [00:36<00:00,  1.03it/s]
Epoch 29/40, Train Loss: 2.9039
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.03s/it]
Epoch 29/40, Validation Loss: 3.1467
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 30/40, Train Loss: 2.9270
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.02it/s]
Epoch 30/40, Validation Loss: 3.1412
Training: 100%|██████████| 38/38 [00:36<00:00,  1.04it/s]
Epoch 31/40, Train Loss: 2.8370
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.08it/s]
Epoch 31/40, Validation Loss: 3.0779
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 32/40, Train Loss: 2.7890
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.00s/it]
Epoch 32/40, Validation Loss: 3.0090
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 33/40, Train Loss: 2.7766
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.11it/s]
Epoch 33/40, Validation Loss: 3.0756
Training: 100%|██████████| 38/38 [00:36<00:00,  1.05it/s]
Epoch 34/40, Train Loss: 2.7316
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.07it/s]
Epoch 34/40, Validation Loss: 2.9935
Training: 100%|██████████| 38/38 [00:35<00:00,  1.06it/s]
Epoch 35/40, Train Loss: 2.7285
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.01s/it]
Epoch 35/40, Validation Loss: 2.9770
Training: 100%|██████████| 38/38 [00:36<00:00,  1.04it/s]
Epoch 36/40, Train Loss: 2.6461
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.10it/s]
Epoch 36/40, Validation Loss: 3.0022
Training: 100%|██████████| 38/38 [00:36<00:00,  1.03it/s]
Epoch 37/40, Train Loss: 2.6796
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.02s/it]
Epoch 37/40, Validation Loss: 2.8705
Training: 100%|██████████| 38/38 [00:35<00:00,  1.06it/s]
Epoch 38/40, Train Loss: 2.6227
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.02s/it]
Epoch 38/40, Validation Loss: 3.1472
Training: 100%|██████████| 38/38 [00:36<00:00,  1.04it/s]
Epoch 39/40, Train Loss: 2.6560
Validation: 100%|██████████| 8/8 [00:07<00:00,  1.11it/s]
Epoch 39/40, Validation Loss: 3.0417
Training: 100%|██████████| 38/38 [00:36<00:00,  1.03it/s]
Epoch 40/40, Train Loss: 2.5472
Validation: 100%|██████████| 8/8 [00:08<00:00,  1.02s/it]
Epoch 40/40, Validation Loss: 2.9211
Testing: 100%|██████████| 8/8 [00:07<00:00,  1.10it/s]Test Accuracy: 0.2812
True label: R, Predicted label: G
True label: R, Predicted label: G
True label: z, Predicted label: L
True label: 5, Predicted label: G
True label: W, Predicted label: m
True label: q, Predicted label: e
True label: L, Predicted label: i
True label: 5, Predicted label: S
True label: Z, Predicted label: Q
True label: Q, Predicted label: W
